# Example Evaluation Code

This notebook will be very __similar__ to the code I use to evaluate your results - it is provided for __your convenience__ so that you can use it to evaluate your preprocessing results at any time before your __final submission__.

Please note that the results here will __NOT__ be the same as my evaluation results.

Let's start with loading the required packages.

In [1]:
# import required package for data handling
import pandas as pd
import numpy as np

# import required packages for splitting data
from sklearn import model_selection
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split

# import required packages for evaluating models
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_recall_fscore_support

# import `logistic regression` model
from sklearn.linear_model import LogisticRegression

Next you should load __your__ data. In this case, I am using a sample dataset (`GroupX.csv`) which contains 6 predictors (`X1 - X6`) and two target variables (`Y1, Y2`).

Please make sure you change the data to your __OWN__ dataset when using this code.

__NOTE__:
1. Your dataset maybe very different from the sample dataset.
2. Please follow this structure when submitting your dataset.

In [2]:
data = pd.read_csv('./Datasets/Kevin22.csv', header=0)
data = data.drop(labels = 'Unnamed: 0', axis=1)
data.head()

,0,1,2,3,4,5,6,7,8,9,...,4.1,5.1,6.1,7.1,8.1,9.1,10.1,11.1,Y1,Y2
0,-0.181588,0.176554,-0.166205,-0.073619,-0.173917,-0.282286,0.126002,0.148107,-0.306138,-0.091122,...,-0.073620,-0.290785,-0.117182,0.381671,0.541550,0.565518,0.437435,-0.032414,0,1
1,0.715586,0.232307,0.774179,-0.073671,-0.179527,-0.297471,0.187801,0.563945,-0.941315,0.353060,...,-0.073672,-0.308516,-0.112598,0.092395,-0.494930,-0.485229,-0.456297,-0.199879,1,0
2,-0.391147,0.130054,-0.386523,-0.073687,-0.186885,-0.308242,0.074778,-0.499583,2.503634,-0.727896,...,-0.073689,-0.308406,-0.124188,1.208455,-0.494930,-0.485229,-0.456297,-0.199879,1,0
3,0.388150,-1.121856,0.200508,-0.073687,-0.186440,-0.306640,-1.195491,-0.192683,2.390660,-0.209258,...,-0.073688,-0.308721,-0.124056,0.114086,-0.494930,-0.485229,-0.456297,-0.199879,1,1
4,-0.456634,-0.506367,-0.495743,-0.063019,0.005073,0.074631,-0.597202,0.082934,-0.391475,0.185601,...,-0.063020,-0.012038,-0.030086,-0.739586,-0.067123,-0.062068,-0.041884,-0.165967,0,1


Checking your data types and make sure it follows the data dictionary would be an important step, you can do that using the `.dtypes` attribute.

__NOTE__: all __continuous__ faetures will be in `float64` data type, and all __categorical__ features will be in `int64` data type (given you already coded (per __suggest task \#6__ in the competition document) them).

In [3]:
data.dtypes

0       float64
1       float64
2       float64
3       float64
4       float64
5       float64
6       float64
7       float64
8       float64
9       float64
10      float64
11      float64
0.1     float64
1.1     float64
2.1     float64
3.1     float64
4.1     float64
5.1     float64
6.1     float64
7.1     float64
8.1     float64
9.1     float64
10.1    float64
11.1    float64
Y1        int64
Y2        int64
dtype: object

Now you need to specify your targets and predictors. __NOTE__ we have two targets here (`Y1, Y2`).

In [4]:
y1 = data.Y1
y2 = data.Y2

Check the shape of the data.

In [5]:
data.shape

(660, 26)

It is very possible that you will use different sets of the predictors for `Y1` and `Y2`. Now let's define them.

First, let's define predictors for `Y1` - which will be the first 5 features in `data`.

In [6]:
cols = list(data.columns)
# first 5 features 
cols[:-2]

['0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 '10',
 '11',
 '0.1',
 '1.1',
 '2.1',
 '3.1',
 '4.1',
 '5.1',
 '6.1',
 '7.1',
 '8.1',
 '9.1',
 '10.1',
 '11.1']

Use below code to select the first 5 features as predictors for `Y1`.

In [7]:
datay1 = data.drop(labels=['0.1','1.1','2.1','3.1','4.1','5.1','6.1','7.1','8.1','9.1','10.1',
                        '11.1','Y1', 'Y2'], axis = 1)

In [8]:
datay1.head()

,0,1,2,3,4,5,6,7,8,9,10,11
0,-0.181588,0.176554,-0.166205,-0.073619,-0.173917,-0.282286,0.126002,0.148107,-0.306138,-0.091122,1.0,0.0
1,0.715586,0.232307,0.774179,-0.073671,-0.179527,-0.297471,0.187801,0.563945,-0.941315,0.353060,0.0,0.0
2,-0.391147,0.130054,-0.386523,-0.073687,-0.186885,-0.308242,0.074778,-0.499583,2.503634,-0.727896,1.0,0.0
3,0.388150,-1.121856,0.200508,-0.073687,-0.186440,-0.306640,-1.195491,-0.192683,2.390660,-0.209258,1.0,0.0
4,-0.456634,-0.506367,-0.495743,-0.063019,0.005073,0.074631,-0.597202,0.082934,-0.391475,0.185601,1.0,0.0


In [9]:

predictors_y1 = datay1

predictors_y1.head()

,0,1,2,3,4,5,6,7,8,9,10,11
0,-0.181588,0.176554,-0.166205,-0.073619,-0.173917,-0.282286,0.126002,0.148107,-0.306138,-0.091122,1.0,0.0
1,0.715586,0.232307,0.774179,-0.073671,-0.179527,-0.297471,0.187801,0.563945,-0.941315,0.353060,0.0,0.0
2,-0.391147,0.130054,-0.386523,-0.073687,-0.186885,-0.308242,0.074778,-0.499583,2.503634,-0.727896,1.0,0.0
3,0.388150,-1.121856,0.200508,-0.073687,-0.186440,-0.306640,-1.195491,-0.192683,2.390660,-0.209258,1.0,0.0
4,-0.456634,-0.506367,-0.495743,-0.063019,0.005073,0.074631,-0.597202,0.082934,-0.391475,0.185601,1.0,0.0


Upon investigation of the data, we know we have __six__ features (`X1 - X6`) predicting `Y2`. Use similar code (as below) to select them.

In [10]:
predictors_y2 = data.drop(['Y1', 'Y2', '0','1','2','3','4','5','6','7','8','9','10','11'], axis=1)
predictors_y2.head()

,0.1,1.1,2.1,3.1,4.1,5.1,6.1,7.1,8.1,9.1,10.1,11.1
0,0.654566,0.525006,-0.151026,0.653205,-0.073620,-0.290785,-0.117182,0.381671,0.541550,0.565518,0.437435,-0.032414
1,-0.619777,-0.497238,0.690584,-0.620570,-0.073672,-0.308516,-0.112598,0.092395,-0.494930,-0.485229,-0.456297,-0.199879
2,0.403480,-0.497238,-0.284283,0.399265,-0.073689,-0.308406,-0.124188,1.208455,-0.494930,-0.485229,-0.456297,-0.199879
3,0.382698,-0.497238,0.636425,0.287623,-0.073688,-0.308721,-0.124056,0.114086,-0.494930,-0.485229,-0.456297,-0.199879
4,-1.266863,-0.037228,-0.494873,-1.176753,-0.063020,-0.012038,-0.030086,-0.739586,-0.067123,-0.062068,-0.041884,-0.165967


Below is the key part of this notebook - which generates a `logistic regression` model to predict `Y1`/`Y2`.

The code works this way:

1. We generate two lists `f1_score_lst` and `auc_lst` to store f1_score and AUC from each of the `10` runs of the model;
2. Define model:
    1. We define a `LogisticRegression()` model;
    
    2. We split predictors (`predictors_y1`) and target `y1` to training (80%) and testing (20%);
    
    3. We fit the model `clf` to the training data, then use it to predict on the testing data;
    
    4. We also defined a `10-fold cross validation` to make sure our model do not overfit - see [here](https://scikit-learn.org/stable/modules/cross_validation.html) for more info;
    
    5. We append the f1_score and AUC of current model to the lists (`f1_score_lst` and `auc_lst`) we defined earlier.
  
3. Print out average f1_score and AUC for all 10 runs;
4. Print out average average accuracy from cross validation
5. Print out confusion matrix and classification report for the __last__ model.

__NOTE__: Step 3 provides the evaluation results we need; step 4 - 5 can be used to verify the results from step 3.

In [11]:
# lists for f1-score and AUC
f1_score_lst = []
auc_lst = []


#loop to calculate f1 and auc scores and present averages after 10 runs
for count in range (1,1000):
    #Model building
    clf = LogisticRegression()
    X1_train, X1_test, y1_train, y1_test = train_test_split(predictors_y1, y1, test_size=0.2, random_state=123)
    clf.fit(X1_train, y1_train)

    y1_pred = clf.predict(X1_test)

    
    #10-fold cross validation
    kfold = model_selection.KFold(n_splits=10, random_state=7)
    scoring = 'accuracy'
    results = model_selection.cross_val_score(clf, X1_train, y1_train, cv=kfold, scoring=scoring)

    

    
    #calculate f1-score and AUC
    
    clf_roc_auc = roc_auc_score(y1_test, y1_pred)
    f1_score_lst.append(precision_recall_fscore_support(y1_test, y1_pred, average='weighted')[2])
    auc_lst.append(clf_roc_auc)


print('F1 {:.4f}; AUC {:.4f} '.format(np.mean(f1_score_lst),np.mean(auc_lst)))

#result=logit_model.fit()
confusion_matrix_y1 = confusion_matrix(y1_test, y1_pred)


#print(result.summary())
print('Accuracy of classifier on test set: {:.2f}'.format(clf.score(X1_test, y1_test)))

print("10-fold cross validation average accuracy of classifier: %.3f" % (results.mean()))

print('Confusion Matrix for Logistic Regression Classfier:')
print(confusion_matrix_y1)

print('Classification Report for Logistic Regression Classfier:')
print(classification_report(y1_test, y1_pred))


F1 0.6768; AUC 0.6759 
Accuracy of classifier on test set: 0.67
10-fold cross validation average accuracy of classifier: 0.600
Confusion Matrix for Logistic Regression Classfier:
[[52 26]
 [17 37]]
Classification Report for Logistic Regression Classfier:
             precision    recall  f1-score   support

          0       0.75      0.67      0.71        78
          1       0.59      0.69      0.63        54

avg / total       0.69      0.67      0.68       132



Below code are used to evaluate model toward `Y2`. It is very similar to the code above - key difference is that `Y2` is imbalanced - so I wrote some code (under `# Begin oversampling`) to deal with that.

In [12]:
# lists for f1-score and AUC
f1_score_lst = []
auc_lst = []


#loop to calculate f1 and auc scores and present averages after 10 runs
for count in range (1,1000):
    #Model building
    clf1 = LogisticRegression()

    
    # Splitting data into testing and training
    X2_train, X2_test, y2_train, y2_test = train_test_split(predictors_y2, y2, test_size=0.2, random_state=123)
    
    # Begin oversampling
    oversample = pd.concat([X2_train,y2_train],axis=1)
    max_size = oversample['Y2'].value_counts().max()
    lst = [oversample]
    for class_index, group in oversample.groupby('Y2'):
        lst.append(group.sample(max_size-len(group), replace=True))
    X2_train = pd.concat(lst)
    y2_train=pd.DataFrame.copy(X2_train['Y2'])
    del X2_train['Y2']
    
    # fitting model on oversampled data
    clf1.fit(X2_train, y2_train)
    
    y2_pred = clf1.predict(X2_test)
    
    
    #10-fold cross validation
    kfold = model_selection.KFold(n_splits=10, random_state=123)
    scoring = 'accuracy'
    results = model_selection.cross_val_score(clf1, X2_train, y2_train, cv=kfold, scoring=scoring)
    
    #calculate f1-score and AUC
    
    clf1_roc_auc = roc_auc_score(y2_test, y2_pred)
    
    
    #calculate average f1-score and AUC
    f1_score_lst.append(precision_recall_fscore_support(y2_test, y2_pred, average='weighted')[2])
    auc_lst.append(clf1_roc_auc)
    
    
print('F1 {:.4f}; AUC {:.4f} '.format(np.mean(f1_score_lst),np.mean(auc_lst)))

confusion_matrix_y2 = confusion_matrix(y2_test, y2_pred)


print('Accuracy of classifier on test set: {:.3f}'.format(clf1.score(X2_test, y2_test)))

print("10-fold cross validation average accuracy of clf1: %.3f" % (results.mean()))

print('Confusion Matrix for Classfier:')
print(confusion_matrix_y2)

print('Classification Report for Classfier:')
print(classification_report(y2_test, y2_pred))


F1 0.6452; AUC 0.6637 
Accuracy of classifier on test set: 0.644
10-fold cross validation average accuracy of clf1: 0.558
Confusion Matrix for Classfier:
[[28  8]
 [39 57]]
Classification Report for Classfier:
             precision    recall  f1-score   support

          0       0.42      0.78      0.54        36
          1       0.88      0.59      0.71        96

avg / total       0.75      0.64      0.66       132

